# **Transformación de tabla physical_specs**
##### **Ejemplo de una carga incremental**

##### **01.- Leer tabla con spark.sql**

In [1]:
%run common_functions

StatementMeta(, 53bcf349-a731-4298-b085-059f4c8b8fc7, 4, Finished, Available, Finished)

In [2]:
df1 = spark.sql("SELECT * FROM lh_silver.dbo.physical_specs")

StatementMeta(, 53bcf349-a731-4298-b085-059f4c8b8fc7, 5, Finished, Available, Finished)

In [3]:
display(df1.head(10))

StatementMeta(, 53bcf349-a731-4298-b085-059f4c8b8fc7, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 09bd8402-fba8-43ba-9b7c-2bd70302d1b1)

##### **02.- Eliminar columna environment**

In [4]:
df2 = df1.drop("environment")

StatementMeta(, 53bcf349-a731-4298-b085-059f4c8b8fc7, 7, Finished, Available, Finished)

In [8]:
df_final = df2.na.replace(
    "",              # valor a buscar
    "Unknown",       # valor de reemplazo
    subset=["cellular_controller"]  # solo en esta columna
)



StatementMeta(, 53bcf349-a731-4298-b085-059f4c8b8fc7, 11, Finished, Available, Finished)

In [9]:
display(df_final.head(10))

StatementMeta(, 53bcf349-a731-4298-b085-059f4c8b8fc7, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 66158c08-ccf5-46d5-802c-54b76ba6be0c)

In [10]:
df_final.printSchema()

StatementMeta(, 53bcf349-a731-4298-b085-059f4c8b8fc7, 13, Finished, Available, Finished)

root
 |-- physical_spec_id: integer (nullable = true)
 |-- width: string (nullable = true)
 |-- height: string (nullable = true)
 |-- depth: string (nullable = true)
 |-- mass: string (nullable = true)
 |-- ram_capacity: string (nullable = true)
 |-- cellular_controller: string (nullable = false)
 |-- battery_capacity: string (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)
 |-- file_date: string (nullable = true)



##### **03.- Escribir datos en lh_gold**

In [11]:
merge_delta_lake(
    table_name="lh_gold.dbo.dim_physical_specs",
    table_path="abfss://Smart_device_Analytics_WS@onelake.dfs.fabric.microsoft.com/lh_gold.Lakehouse/Tables/dbo/dim_physical_specs",
    merge_condition= "tgt.physical_spec_id = src.physical_spec_id AND tgt.file_date = src.file_date",
    input_df= df_final,
    partition_column= "file_date"
)


StatementMeta(, 53bcf349-a731-4298-b085-059f4c8b8fc7, 14, Finished, Available, Finished)

In [12]:
%%sql
select file_date, count(1) from dim_physical_specs
group by file_date;

StatementMeta(, 53bcf349-a731-4298-b085-059f4c8b8fc7, 15, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 2 fields>